In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import  LabelEncoder
from scipy.stats import chi2_contingency

In [ ]:
# Veri yükleme fonksiyonu
def load_data(file_path):
    return pd.read_excel(file_path)

In [ ]:
# Eksik verileri görselleştirme fonksiyonu
def visualize_missing_data(df):
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
    plt.title('Eksik Veriler Haritası')
    plt.show()

In [ ]:
# Aykırı değerleri görselleştirme fonksiyonu
def visualize_outliers(df):
    plt.figure(figsize=(12, 6))
    for i, col in enumerate(df.select_dtypes(include=[np.number]).columns, 1):
        plt.subplot(3, 4, i)
        sns.boxplot(df[col])
        plt.title(f'{col} - Kutu Grafiği')
    plt.tight_layout()
    plt.show()

In [ ]:
# Kategorik verileri görselleştirme fonksiyonu
def plot_categorical_distribution(df, column, n=10):
    def top_n_categories(series, n):
        top_categories = series.value_counts().nlargest(n).index
        return series.apply(lambda x: x if x in top_categories else 'Diğer')
    
    temp_df = df.copy()
    if temp_df[column].nunique() > 20:
        temp_df[column + '_Top'] = top_n_categories(temp_df[column], n=n)
        plt.figure(figsize=(12, 6))
        ax = sns.countplot(data=temp_df, x=column + '_Top', order=temp_df[column + '_Top'].value_counts().index)
        plt.title(f'{column} - En Sık Görülen {n} Kategori')
    else:
        plt.figure(figsize=(12, 6))
        ax = sns.countplot(data=temp_df, x=column, order=temp_df[column].value_counts().index)
        plt.title(f'{column} - Dağılımı')
    
    plt.xticks(rotation=45, ha='right')
    plt.show()

In [ ]:
# Veri çerçevesinin temel bilgilerini gösteren fonksiyon
def display_basic_info(df):
    print(f"Veri Kümesi Boyutu: {df.shape}")
    print("Değişken İsimleri:", df.columns)
    print("Veri Tipleri:", df.dtypes)
    print("Boş Değer Sayıları:", df.isnull().sum())
    print("\nVeri çerçevesinin bilgileri:")
    print(df.info())
    print("\nTemel istatistikler:")
    print(df.describe())

In [ ]:
# Chi-Square testi fonksiyonu
def chi_square_test(df, col1, col2):
    kontenjans_tablosu = pd.crosstab(df[col1], df[col2])
    chi2_stat, p_value, dof, expected = chi2_contingency(kontenjans_tablosu)
    return chi2_stat, p_value

In [ ]:
# Veri temizleme ve doldurma işlemleri
def clean_and_fill_data(df):
    simpleimputer = SimpleImputer(strategy='most_frequent')
    df['Cinsiyet'] = df.groupby('Il')['Cinsiyet'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Belirtilmemiş'))
    
    filled_values = simpleimputer.fit_transform(df[['Il']])
    df['Il'] = pd.DataFrame(filled_values, columns=['Il'])
    
    df[['Alerjilerim', 'Kronik Hastaliklarim']] = df[['Alerjilerim', 'Kronik Hastaliklarim']].fillna('Bilinmiyor')
    
    filled_values = simpleimputer.fit_transform(df[['Kan Grubu']])
    df["Kan Grubu"] = pd.DataFrame(filled_values, columns=["Kan Grubu"])
    
    imputer = KNNImputer(n_neighbors=5)
    filled_values = imputer.fit_transform(df[['Boy', 'Kilo', 'Yas']])
    df[['Boy', 'Kilo', 'Yas']] = pd.DataFrame(filled_values, columns=['Boy', 'Kilo', 'Yas'])
    
    return df

In [ ]:
# Kronik hastalıkları kategorize etme fonksiyonu
def categorize_chronic_diseases(df):
    def kronik_hastalik_kategorileri(x):
        kronik_hastalıklar = ['Hipertansiyon', 'Kan Hastalıkları', 'Diyabet', 'KOAH', 'Alzheimer', 'Kanser', 'Guatr', 'Kalp Hastalıkları', 'Astim', 'Kemik Erimesi', 'Diger']
        kategoriler = [hastalık for hastalık in kronik_hastalıklar if hastalık in str(x)]
        return ', '.join(kategoriler) if kategoriler else 'Bilinmiyor'

    for col in ['Kronik Hastaliklarim', 'Baba Kronik Hastaliklari', 'Anne Kronik Hastaliklari', 'Kiz Kardes Kronik Hastaliklari', 'Erkek Kardes Kronik Hastaliklari']:
        df[col] = df[col].apply(kronik_hastalik_kategorileri)
    
    return df

In [ ]:
# İlaç adlarını kategorize etme fonksiyonu
def categorize_medication_names(df):
    top_30_ilaclar = df['Ilac_Adi'].value_counts().nlargest(30).index
    df['Ilac_Adi'] = df['Ilac_Adi'].apply(lambda x: x if x in top_30_ilaclar else 'Diğer')
    return df

In [ ]:
# Kilo, boy ve yaş kategorize etme fonksiyonları
def categorize_weight(weight):
    if weight <= 60:
        return 'Düşük'
    elif weight <= 80:
        return 'Orta'
    else:
        return 'Yüksek'

In [ ]:
def categorize_height(height):
    if height <= 160:
        return 'Kısa'
    elif height <= 180:
        return 'Orta'
    else:
        return 'Uzun'

In [ ]:
def categorize_age(age):
    if age <= 12:
        return 'Çocuk'
    elif age <= 24:
        return 'Genç'
    elif age <= 64:
        return 'Yetişkin'
    else:
        return 'Yaşlı'

In [ ]:
# Kategorileştirme fonksiyonlarını uygulama
def apply_categorization(df):
    df['Kilo'] = df['Kilo'].apply(categorize_weight)
    df['Boy'] = df['Boy'].apply(categorize_height)
    df['Yas'] = df['Yas'].apply(categorize_age)
    return df

In [ ]:
# Ana işlem akışı
def main_pipeline(file_path):
    df = load_data(file_path)
    visualize_missing_data(df)
    visualize_outliers(df)
    display_basic_info(df)
    
    # Kategorik sütunların dağılım grafiğini oluşturma
    categorical_columns = df.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        plot_categorical_distribution(df, col)

    # Veri temizleme
    df = clean_and_fill_data(df)
    
    # Kronik hastalık ve ilaç adlarını kategorize etme
    df = categorize_chronic_diseases(df)
    df = categorize_medication_names(df)
    
    # Yaş hesaplama
    df['Yas'] = 2024 - df['Dogum_Tarihi'].dt.year
    
    # Kategorileştirme
    df = apply_categorization(df)
    
    # Kategorik değişkenleri sayısal forma dönüştürme
    categorical_columns = [
        'Cinsiyet', 'Il', 'Ilac_Adi', 'Alerjilerim', 'Kronik Hastaliklarim',
        'Baba Kronik Hastaliklari', 'Anne Kronik Hastaliklari',
        'Kiz Kardes Kronik Hastaliklari', 'Erkek Kardes Kronik Hastaliklari',
        'Kan Grubu', 'Yas', 'Kilo', 'Boy'
    ]
    
    for column in categorical_columns:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
    
    # Chi-Square testi örnekleri
    test_columns = [('Cinsiyet', 'Kronik Hastaliklarim'), ('Cinsiyet', 'Yan_Etki'), ('Alerjilerim', 'Il')]
    results = []
    
    for col1, col2 in test_columns:
        chi2_stat, p_value = chi_square_test(df, col1, col2)
        bağımsızlık = 'Bağımsız' if p_value >= 0.05 else 'Bağımsız Değil'
        results.append({'Sütun1': col1, 'Sütun2': col2, 'Chi2 Stat': chi2_stat, 'P Değeri': p_value, 'Bağımsızlık': bağımsızlık})

    sonuç_df = pd.DataFrame(results)
    print(sonuç_df)

In [ ]:
# Programı çalıştır
main_pipeline('side_effect_data.xlsx')